In [1]:
# Setup: Configure Python path and database location
# This ensures the notebook can import astrofiler modules and connect to the DB

import os
import sys
from pathlib import Path

# Determine repository root (handles running from notebooks/ or repo root)
REPO_ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name == 'notebooks' else Path.cwd().resolve()
SRC = REPO_ROOT / 'src'

# Add src/ to Python path so we can import astrofiler modules
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# Point Peewee ORM at the repo's astrofiler.db by default
# Override by setting ASTROFILER_DB_PATH env var before importing models
DEFAULT_DB = REPO_ROOT / 'astrofiler.db'
os.environ.setdefault('ASTROFILER_DB_PATH', str(DEFAULT_DB))

print('Repo root:', REPO_ROOT)
print('DB path  :', os.environ.get('ASTROFILER_DB_PATH'))
print('DB exists:', Path(os.environ['ASTROFILER_DB_PATH']).exists())

Repo root: D:\Dropbox\Projects\astrofiler-gui-dev
DB path  : D:\Dropbox\Projects\astrofiler-gui-dev\astrofiler.db
DB exists: True


In [2]:
# Load variable-star targets from AstroFiler database
# These are targets marked via the Images view "Add Variable Star" context menu

from astrofiler.database import DatabaseManager
from astrofiler.models import db, VariableStars

# Ensure database schema is up to date (runs pending migrations + creates tables)
mgr = DatabaseManager(db)
mgr.setup_database()

# Query all designated variable-star targets
db.connect(reuse_if_open=True)
try:
    stars = [vs.target_name for vs in VariableStars.select().order_by(VariableStars.target_name)]
finally:
    db.close()

print(f'VariableStars count: {len(stars)}')
for s in stars:
    print(' -', s)

VariableStars count: 1
 - AG_DRA


## Choose target + stacked FITS

Set `STAR_NAME` to one of the names printed above (or any resolvable name).
Set `STACKED_FITS_PATH` to a photometric stack FITS with WCS.

Tip: The file-picker helper below is optional and uses `tkinter` (works on most local Python installs).

In [3]:
# Configuration: Set target star and path to stacked FITS
# Modify these variables to match your observation

STAR_NAME = stars[0] if stars else 'RW Aur'  # <- change as desired
STACKED_FITS_PATH = ''  # <- set this manually, or use the picker below

print('STAR_NAME:', STAR_NAME)
print('STACKED_FITS_PATH:', STACKED_FITS_PATH or '(not set)')

STAR_NAME: AG_DRA
STACKED_FITS_PATH: (not set)


In [11]:
# Query database for sessions matching the selected target
# This helps you find available observation sessions for the chosen variable star

from astrofiler.models import fitsSession, fitsFile
from datetime import datetime

# Connect to database and query sessions for the target
db.connect(reuse_if_open=True)
try:
    # Query sessions where the object name matches the selected star
    matching_sessions = list(
        fitsSession.select()
        .where(fitsSession.fitsSessionObjectName == STAR_NAME)
        .order_by(fitsSession.fitsSessionDate.desc())
    )
finally:
    db.close()

print(f'Found {len(matching_sessions)} session(s) for target "{STAR_NAME}":\n')

# Display sessions with key details
session_list = []
for i, session in enumerate(matching_sessions, 1):
    session_date = session.fitsSessionDate if session.fitsSessionDate else 'Unknown'
    session_id = session.fitsSessionId if session.fitsSessionId else '?'
    filter_band = session.fitsSessionFilter if session.fitsSessionFilter else 'Unknown'
    
    # Count files in this session
    db.connect(reuse_if_open=True)
    try:
        frame_count = fitsFile.select().where(
            (fitsFile.fitsFileSession == session_id) & 
            (fitsFile.fitsFileType == 'Light Frame')
        ).count()
    finally:
        db.close()
    
    print(f'{i}. Session ID: {session_id}')
    print(f'   Date: {session_date}')
    print(f'   Filter: {filter_band}')
    print(f'   Light frames: {frame_count}')
    print()
    
    session_list.append({
        'index': i,
        'id': session_id,
        'date': session_date,
        'filter': filter_band,
        'frame_count': frame_count,
        'session_obj': session
    })

# Instructions for user
if session_list:
    print('To use a specific session, set SESSION_INDEX to the number above (e.g., SESSION_INDEX = 1)')
    print('The next cell will help you find or create a stacked FITS for the selected session.')
else:
    print('No sessions found. You may need to:')
    print('  1. Ensure the target name matches exactly what\'s in the database')
    print('  2. Have already imported/registered observations for this target')
    print('  3. Set STACKED_FITS_PATH manually if you have an existing stack')

# Default: select first session if available
SESSION_INDEX = 1 if session_list else None
SELECTED_SESSION = session_list[SESSION_INDEX - 1]['session_obj'] if SESSION_INDEX and session_list else None

if SELECTED_SESSION:
    print(f'\nDefaulted to SESSION_INDEX = {SESSION_INDEX}')
    print(f'Selected session: {session_list[SESSION_INDEX - 1]["date"]} (ID: {session_list[SESSION_INDEX - 1]["id"]})')

Found 1 session(s) for target "AG_DRA":

1. Session ID: 1e07c35b-6165-45b5-9c92-107834999cd8
   Date: 2020-06-23
   Filter: RGB
   Light frames: 0

To use a specific session, set SESSION_INDEX to the number above (e.g., SESSION_INDEX = 1)
The next cell will help you find or create a stacked FITS for the selected session.

Defaulted to SESSION_INDEX = 1
Selected session: 2020-06-23 (ID: 1e07c35b-6165-45b5-9c92-107834999cd8)


In [21]:
# Find the stacked FITS file for the selected session
# Uses the same logic as the Sessions view to locate stacked files

import os
import glob
from pathlib import Path

if not SELECTED_SESSION:
    print('No session selected. Set SESSION_INDEX to a valid session number, or set STACKED_FITS_PATH manually.')
else:
    session_id = SELECTED_SESSION.fitsSessionId
    session_date = SELECTED_SESSION.fitsSessionDate
    filter_band = SELECTED_SESSION.fitsSessionFilter or 'RGB'
    object_name = SELECTED_SESSION.fitsSessionObjectName or 'Unknown'
    
    print(f'Searching for stacked FITS for session: {session_id}')
    print(f'Target: {object_name}, Filter: {filter_band}, Date: {session_date}\n')
    
    # Get directory where session files are stored (same as Sessions view logic)
    db.connect(reuse_if_open=True)
    try:
        sample_file = fitsFile.select().where(
            (fitsFile.fitsFileSession == session_id) &
            (fitsFile.fitsFileType == 'Light Frame') &
            (fitsFile.fitsFileSoftDelete == False)
        ).first()
    finally:
        db.close()
    

Searching for stacked FITS for session: 1e07c35b-6165-45b5-9c92-107834999cd8
Target: AG_DRA, Filter: RGB, Date: 2020-06-23

Cannot find session directory: no light frames found.
Ensure you have imported observations for this session.

Current STACKED_FITS_PATH: (not set)


## Run variable-star photometry

This runs the complete AstroFiler photometry workflow:
- Resolve target coordinates via Astropy's name resolver
- Download AAVSO VSP comparison stars for the field
- SEP source extraction (background subtraction + detection)
- Match catalog stars to detected sources (WCS-based matching)
- SEP circular aperture sums for matched sources
- Ensemble linear fit: instrumental magnitudes → catalog magnitudes
- Calculate target magnitude from the fit

In [12]:
# Execute the variable-star photometry pipeline
# This calls the core workflow that implements the RWAUR notebook logic

from pathlib import Path

from astrofiler.core.variable_star_photometry import (
    VariableStarPhotometryOptions,
    run_variable_star_photometry,
)

# Validate inputs
if not STACKED_FITS_PATH:
    raise ValueError('Set STACKED_FITS_PATH to a stacked FITS file')
if not Path(STACKED_FITS_PATH).exists():
    raise FileNotFoundError(STACKED_FITS_PATH)

# Configure photometry options
opts = VariableStarPhotometryOptions(
    band='V',                        # Photometric band (V, B, R, I, etc.)
    field_of_view_arcmin=18.5,       # FOV for AAVSO VSP query
    brightest_comp_mag=11.0,         # Ensemble fit magnitude range
    dimmest_comp_mag=13.0,
    source_snr=20.0,                 # SEP detection threshold (SNR)
    match_radius_arcsec=4.0,         # Catalog-to-source matching radius
    aperture_radius_pixels=6.0,      # Photometry aperture radius
    bkg_subtract=True,               # SEP background subtraction
)

# Run the photometry pipeline
result = run_variable_star_photometry(
    STACKED_FITS_PATH,
    star_name=STAR_NAME,
    options=opts,
    check_auid=None,  # optionally set an AUID present in the chart as a check star
)

# Display summary
summary = {k: result.get(k) for k in [
    'star_name', 'band', 'chart_id', 'date_obs',
    'target_ra_deg', 'target_dec_deg',
    'target_instrumental_mag', 'target_magnitude',
    'check_auid', 'check_magnitude'
]}
summary

ValueError: Set STACKED_FITS_PATH to a stacked FITS file

In [ ]:
# Inspect the photometry results: ensemble comparison stars and matched sources
# This shows which AAVSO comps were used in the fit and displays sample measurements

print('Ensemble AUIDs:', result.get('ensemble_auids'))

# Show first few matched rows (AAVSO comps + target)
rows = result.get('rows') or []
print(f'\nTotal matched sources: {len(rows)}')
print('\nFirst 10 sources:')
for r in rows[:10]:
    print({
        'auid': r.get('auid'),
        'vmag': r.get('vmag'),
        'instrumental_mag': r.get('instrumental_mag'),
        'aperture_sum': r.get('aperture_sum'),
        'x': r.get('x'),
        'y': r.get('y'),
    })

## Generate AAVSO-Ready Report Bundle

This creates a submission-ready output package next to the stacked FITS:
- `aavso_report_<star>_<timestamp>.json` – Full structured output (all measurements)
- `aavso_rows_<star>_<timestamp>.csv` – Per-source data (all matched stars including target)
- `aavso_upload_template_<star>_<timestamp>.txt` – Human-readable summary for AAVSO WebObs

**Note:** AAVSO WebObs upload formats vary. Use the JSON/CSV to transform into the required format if needed.

In [ ]:
# Write AAVSO report bundle: JSON + CSV + upload template
# This creates three files next to your stacked FITS for easy submission

import csv
import json
from datetime import datetime, timezone

from astropy.time import Time

# Determine output directory and generate timestamp
stack_path = Path(result['stacked_fits']).resolve()
out_dir = stack_path.parent
ts = datetime.now(timezone.utc).strftime('%Y%m%dT%H%M%SZ')
safe_star = ''.join(ch for ch in result['star_name'] if ch.isalnum() or ch in ('-', '_')).strip('_')

# File paths
report_json = out_dir / f"aavso_report_{safe_star}_{ts}.json"
rows_csv = out_dir / f"aavso_rows_{safe_star}_{ts}.csv"
upload_txt = out_dir / f"aavso_upload_template_{safe_star}_{ts}.txt"

# Compute Julian Date if DATE-OBS is available
jd = None
if result.get('date_obs'):
    try:
        jd = float(Time(result['date_obs']).jd)
    except Exception:
        jd = None

# Prepare full report bundle
bundle = dict(result)
bundle['jd'] = jd

# Write JSON report (complete structured output)
report_json.write_text(json.dumps(bundle, indent=2, sort_keys=True), encoding='utf-8')

# Write per-star CSV (all matched sources with measurements)
rows = bundle.get('rows') or []
fieldnames = sorted({k for r in rows for k in r.keys()}) if rows else []
with rows_csv.open('w', newline='', encoding='utf-8') as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    for r in rows:
        w.writerow(r)

# Write human-readable upload template (for AAVSO WebObs / notes)
template_lines = [
    '# AAVSO Upload Template',
    '# (Fill in observer details and format for your submission method)',
    '',
    f"Star: {bundle.get('star_name')}",
    f"Band: {bundle.get('band')}",
    f"Chart ID: {bundle.get('chart_id')}",
    f"DATE-OBS: {bundle.get('date_obs')}",
    f"Julian Date: {bundle.get('jd')}",
    f"Magnitude: {bundle.get('target_magnitude')}",
    f"Check AUID: {bundle.get('check_auid')}",
    f"Check Magnitude: {bundle.get('check_magnitude')}",
    f"Ensemble AUIDs: {', '.join(bundle.get('ensemble_auids', []))}",
    '',
    '## Notes:',
    '- This is an AstroFiler-generated estimate using ensemble photometry against AAVSO VSP comps.',
    '- SEP was used for source detection, centroiding, and aperture photometry.',
    '- If WebObs requires a specific file format, use the JSON/CSV to generate it.',
    '- Consider adding uncertainty/error estimation before submission if required.',
]
upload_txt.write_text('\n'.join(template_lines), encoding='utf-8')

print('Wrote AAVSO report bundle:')
print(' -', report_json)
print(' -', rows_csv)
print(' -', upload_txt)